In [1]:
# create a dataframe
import boto3
import io
import pandas as pd
import time
from io import StringIO
import numpy as np

In [2]:
def process_partD(path, year):
    '''
    Input: source path to raw file and year
    Process: import table as a pd.DatafFrame and add `year` column
    Output: pd.DatafFrame
    '''
    chunksize = 10 ** 6
    chunks_df = []
    for chunk in pd.read_csv(path, sep="\t", low_memory=False, chunksize=chunksize):
        chunks_df.append(chunk)
    df = pd.concat(chunks_df, axis=0)
    # append a year column
    df['year'] = pd.Series([year] * len(df))
    return df

In [3]:
s3 = boto3.client('s3')
bucket='dast1healthcare' # Or whatever you called your bucket

path_tuples = [("PartD_Prescriber_PUF_NPI_Drug_13.txt", 2013),
               ("PartD_Prescriber_PUF_NPI_Drug_14.txt", 2014),
               ("PartD_Prescriber_PUF_NPI_Drug_15.txt", 2015),
               ("PartD_Prescriber_PUF_NPI_Drug_16.txt", 2016)]

df_list = []
for path in path_tuples:
    data_key = path[0] # Where the file is within your bucket
    data_location = 's3://{}/{}'.format(bucket, data_key)
    df_list.append(process_partD(data_location, path[1]))
df = pd.concat(df_list, axis=0)

In [23]:
'''
# save the huge merged part D data frame
csv_buffer = StringIO()
df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'medicare_part_d.csv').put(Body=csv_buffer.getvalue())
'''

"\n# save the huge merged part D data frame\ncsv_buffer = StringIO()\ndf.to_csv(csv_buffer)\ns3_resource = boto3.resource('s3')\ns3_resource.Object(bucket, 'medicare_part_d.csv').put(Body=csv_buffer.getvalue())\n"

In [4]:
# find unique ids
unique_id = np.array(list(set(zip(df["npi"].values,
                                  df["nppes_provider_last_org_name"].values, 
                                  df["nppes_provider_first_name"].values,
                                  df["specialty_description"].values,
                                  df["nppes_provider_city"].values,
                                  df["nppes_provider_state"].values)))).T
id_df = pd.DataFrame(unique_id.T, columns=["npi","last/org name","first name","specialty","city","state"])

In [25]:
'''
# save unique ids file
csv_buffer = StringIO()
id_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'identification_part_d.csv').put(Body=csv_buffer.getvalue())
'''

"\n# save unique ids file\ncsv_buffer = StringIO()\nid_df.to_csv(csv_buffer)\ns3_resource = boto3.resource('s3')\ns3_resource.Object(bucket, 'identification_part_d.csv').put(Body=csv_buffer.getvalue())\n"

---
## EDA: what will the a feature matrix look like

### What's going to be in the rows?

In [5]:
unique_ids = list(set(zip(*id_df.drop(columns=['npi']).as_matrix().T.tolist())))
unique_names = list(set(zip(*id_df[['last/org name','first name']].as_matrix().T.tolist())))
print("length of unique items: ",id_df.shape[0])
print("length of unique ids: ",len(unique_ids))
print("length of unique names: ",len(unique_names))
print("length of unique npi: ", len(set(id_df['npi'].values)))

length of unique items:  1296170
length of unique ids:  1295207
length of unique names:  907377
length of unique npi:  1075934


**We're going to go with unique npis in the rows for now!**

In [6]:
df1 = df.drop(columns=['nppes_provider_last_org_name',
                       'nppes_provider_first_name',
                       'nppes_provider_city',
                       'nppes_provider_state',
                       'specialty_description'])

In [7]:
df1.head()

,npi,description_flag,drug_name,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
0,1003000126,S,ISOSORBIDE MONONITRATE ER,ISOSORBIDE MONONITRATE,NaN,11,11.0,307,171.59,NaN,*,NaN,*,NaN,NaN,NaN,2013
1,1003000126,S,LEVOFLOXACIN,LEVOFLOXACIN,26.0,26,26.0,165,227.10,15.0,NaN,15.0,NaN,15.0,106.0,159.72,2013
2,1003000126,S,LISINOPRIL,LISINOPRIL,17.0,19,19.0,570,100.37,NaN,#,NaN,#,NaN,NaN,NaN,2013
3,1003000126,S,METOPROLOL TARTRATE,METOPROLOL TARTRATE,28.0,30,31.0,916,154.65,NaN,#,NaN,#,NaN,NaN,NaN,2013
4,1003000126,S,PREDNISONE,PREDNISONE,14.0,14,14.0,133,44.72,NaN,*,NaN,*,NaN,NaN,NaN,2013


### What's going to be in the columns?

In [8]:
unique_generic_names = list(set(df1["generic_name"].values))
unique_drug_names = list(set(df1["drug_name"].values))
print("number of unique drugs by generic_name: ", len(unique_generic_names))
print("number of unique drugs by drug_name: ", len(unique_drug_names))

number of unique drugs by generic_name:  2000
number of unique drugs by drug_name:  3394


**We're going to go with generic_name in the columns for now**

In [9]:
df2 = df1.drop(columns=['drug_name'])
df2.head()

,npi,description_flag,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
0,1003000126,S,ISOSORBIDE MONONITRATE,NaN,11,11.0,307,171.59,NaN,*,NaN,*,NaN,NaN,NaN,2013
1,1003000126,S,LEVOFLOXACIN,26.0,26,26.0,165,227.10,15.0,NaN,15.0,NaN,15.0,106.0,159.72,2013
2,1003000126,S,LISINOPRIL,17.0,19,19.0,570,100.37,NaN,#,NaN,#,NaN,NaN,NaN,2013
3,1003000126,S,METOPROLOL TARTRATE,28.0,30,31.0,916,154.65,NaN,#,NaN,#,NaN,NaN,NaN,2013
4,1003000126,S,PREDNISONE,14.0,14,14.0,133,44.72,NaN,*,NaN,*,NaN,NaN,NaN,2013


In [31]:
description_flags = list(set(df2['description_flag'].values))
bene_count_ge65_suppress_flags = list(set(df2['bene_count_ge65_suppress_flag'].values))
ge65_suppress_flags = list(set(df2['ge65_suppress_flag'].values))
print('description_flags: ', description_flags)
print('bene_count_ge65_suppress_flags: ', bene_count_ge65_suppress_flags)
print('ge65_suppress_flags: ', ge65_suppress_flags)

description_flags:  ['T', 'S']
bene_count_ge65_suppress_flags:  [nan, '#', '*']
ge65_suppress_flags:  [nan, '#', '*']


**We're going to drop `description_flag`,`bene_count_ge65_suppress_flags`,and `ge65_suppress_flags`.**

In [10]:
df3 = df2.drop(columns=['description_flag','bene_count_ge65_suppress_flag','ge65_suppress_flag'])
df3.head()

,npi,generic_name,bene_count,total_claim_count,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,total_claim_count_ge65,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65,year
0,1003000126,ISOSORBIDE MONONITRATE,NaN,11,11.0,307,171.59,NaN,NaN,NaN,NaN,NaN,2013
1,1003000126,LEVOFLOXACIN,26.0,26,26.0,165,227.10,15.0,15.0,15.0,106.0,159.72,2013
2,1003000126,LISINOPRIL,17.0,19,19.0,570,100.37,NaN,NaN,NaN,NaN,NaN,2013
3,1003000126,METOPROLOL TARTRATE,28.0,30,31.0,916,154.65,NaN,NaN,NaN,NaN,NaN,2013
4,1003000126,PREDNISONE,14.0,14,14.0,133,44.72,NaN,NaN,NaN,NaN,NaN,2013


### Let's start building the matrix
> 1. merge npi and year
> 2. build feature matrix
> 3. split feature matrix by year

In [11]:
# merging npi and year
df3['year'] = pd.to_numeric(df3['year'])
df3['year_npi'] = list(zip(*df3[['year','npi']].as_matrix().T.astype(str)))

# drop year and npi
df4 = df3.drop(columns=["npi","year"])
del df, df1, df2, df3

In [35]:
# transform dataframe
df = df4[['year_npi','generic_name',
          'bene_count',
          'total_claim_count',
          'total_30_day_fill_count',
          'total_day_supply',
          'total_drug_cost',
          'bene_count_ge65',
          'total_claim_count_ge65',
          'total_30_day_fill_count_ge65',
          'total_day_supply_ge65',
          'total_drug_cost_ge65']].groupby(['year_npi','generic_name']).agg('sum').reset_index()

In [39]:
df_bene_count = df.pivot(index='year_npi', columns='generic_name', values='bene_count')
df_bene_count.shape

(3405384, 2000)

In [41]:
df_bene_count.

Index                             27243072
0.9 % SODIUM CHLORIDE             27243072
AA 3%/ELECTROLYTE-TPN SOLN/GLY    27243072
AA 3.31 %/D9.8W/FAT/ELECT #10     27243072
AA 4.25%/CALCIUM/LYTES/D25W       27243072
AA 4.25%/CALCIUM/LYTES/D5W        27243072
AA 4.25%/CALCIUM/LYTES/DEX 10%    27243072
AA 5 %/CALCIUM/LYTES/DEXT 15 %    27243072
AA 5 %/CALCIUM/LYTES/DEXT 20 %    27243072
AA 5 %/CALCIUM/LYTES/DEXT 25 %    27243072
ABACAVIR SULFATE                  27243072
ABACAVIR SULFATE/LAMIVUDINE       27243072
ABACAVIR/DOLUTEGRAVIR/LAMIVUDI    27243072
ABACAVIR/LAMIVUDINE/ZIDOVUDINE    27243072
ABATACEPT                         27243072
ABATACEPT/MALTOSE                 27243072
ABIRATERONE ACETATE               27243072
ABOBOTULINUMTOXINA                27243072
ACAMPROSATE CALCIUM               27243072
ACARBOSE                          27243072
ACEBUTOLOL HCL                    27243072
ACETAMINOPHEN WITH CODEINE        27243072
ACETAMINOPHEN/CAFF/DIHYDROCOD     27243072
ACETAMINOPH

In [45]:
mat_size = df_bene_count.memory_usage()[0] * df_bene_count.shape[1]
print('bene_count sparse matrix size: {} GiB'.format(round(mat_size/(10**9),1)))

bene_count sparse matrix size: 54.5 GiB


**We have 10 distict explicit data fields from Medicare Part D and we can also, derive the following implicit data fields:**
> - Drug cost per beneficiary
> - Drug cost per claim
> - Drug cost per beneficiary ge_65
> - Drug cost per claim ge_65
> - Relative distribution of drug cost as a percentage of total

**_All in, that's about 15 data fields or 817.5 GiB of sparse matrices!!!_**

---
## Start Building the Machine Learning Model

### ML Modeling Pipeline:
> 1. Build out all explicit and implicit feature matrices
> 2. Train models using regressors for each feature matrix
> 3. Pass probabilities from each model into a master matrix
> 4. Train master model

**_Given that each feature matrix is for a specific data field, I expect the results of the model to be highly interpretable_**

> **Concerns:** There are 3,405,384 unique records (year_npi) and only 2,879 targets (excluded individuals with NPI numbers) over the same period.<br>
> **Solution Strategy:** Perform a Chi-Squared test to determine how independant are the 15 features from one another.

In [ ]:
'''
# save unique ids file
start_time = time.time()
csv_buffer = StringIO()
df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'bene_count.csv').put(Body=csv_buffer.getvalue())

del df
end_time = time.time()
print("time elapsed: ", end_time-start_time)
'''

In [132]:
'''
bene_count_df = df4[['year_npi','generic_name','bene_count']]
total_claim_count_df = df4[['year_npi','generic_name','total_claim_count']]
total_30_day_fill_count_df = df4[['year_npi','generic_name','total_30_day_fill_count']]
total_day_supply_df = df4[['year_npi','generic_name','total_day_supply']]
total_drug_cost_df = df4[['year_npi','generic_name','total_drug_cost']]
bene_count_ge65_df = df4[['year_npi','generic_name','bene_count_ge65']]
total_claim_count_ge65_df = df4[['year_npi','generic_name','total_claim_count_ge65']]
total_30_day_fill_count_ge65_df = df4[['year_npi','generic_name','total_30_day_fill_count_ge65']]
total_day_supply_ge65_df = df4[['year_npi','generic_name','total_day_supply_ge65']]
total_drug_cost_ge65_df = df4[['year_npi','generic_name','total_drug_cost_ge65']]
'''

MemoryError: 

# df4.head()

In [ ]:
df3.head()

In [20]:
df.shape[0]-df3[pd.isnull(df3['year'])].shape[0]

4000000

NameError: name 'df2013' is not defined